In [1]:
import io
import time
import requests
import datetime
import pandas as pd

from util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'
path_data = './data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,0,0,0,NaN,NaN,0,5,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,0,0,0,NaN,NaN,0,0,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,0,0,0,NaN,NaN,0,1,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,0,0,0,NaN,NaN,0,10,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,18,0,0,NaN,NaN,18,148,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5497,2020-11-11T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1376,202,1578,21986,...,732,1487,10656,762,21594.0,13388.0,34982,780247,541877.0,NaN
5498,2020-11-11T17:00:00,ITA,9,Toscana,43.769231,11.255889,1628,246,1874,46026,...,1586,2507,20319,1701,56467.0,13453.0,69920,1259794,822966.0,effettuati 34960 tamponi rapidi antigenici di...
5499,2020-11-11T17:00:00,ITA,10,Umbria,43.106758,12.388247,359,68,427,9907,...,194,515,5701,218,4597.0,11656.0,16253,340853,194467.0,NaN
5500,2020-11-11T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,160,17,177,2155,...,33,125,2031,225,4052.0,536.0,4588,47373,28237.0,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5497,2020-11-11T17:00:00,19,Sicilia,1376,202,1578,21986,23564,732,1487,10656,762,21594.0,13388.0,34982,780247,541877.0
5498,2020-11-11T17:00:00,9,Toscana,1628,246,1874,46026,47900,1586,2507,20319,1701,56467.0,13453.0,69920,1259794,822966.0
5499,2020-11-11T17:00:00,10,Umbria,359,68,427,9907,10334,194,515,5701,218,4597.0,11656.0,16253,340853,194467.0
5500,2020-11-11T17:00:00,2,Valle d'Aosta,160,17,177,2155,2332,33,125,2031,225,4052.0,536.0,4588,47373,28237.0


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2020-11-11', '2020-11-10')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-11T17:00:00,1,Piemonte,4790,340,5130,55224,60354,1519,2953,42087,4826,45698.0,61569.0,107267,1191915,755259.0
1,2020-11-11T17:00:00,2,Valle d'Aosta,160,17,177,2155,2332,33,125,2031,225,4052.0,536.0,4588,47373,28237.0
2,2020-11-11T17:00:00,3,Lombardia,6907,764,7671,134247,141918,6413,8180,124025,18723,221239.0,63427.0,284666,3377673,2066395.0
3,2020-11-11T17:00:00,5,Veneto,1669,208,1877,50358,52235,1985,3082,32413,2689,26526.0,60811.0,87337,2494150,965761.0
4,2020-11-11T17:00:00,6,Friuli Venezia Giulia,346,46,392,7773,8165,324,572,7448,489,13960.0,2142.0,16102,595748,246668.0
5,2020-11-11T17:00:00,7,Liguria,1315,98,1413,12542,13955,619,1102,23278,2009,27482.0,11760.0,39242,503245,256277.0
6,2020-11-11T17:00:00,8,Emilia-Romagna,2072,222,2294,41597,43891,2111,2428,29516,4876,52446.0,25837.0,78283,1784161,939211.0
7,2020-11-11T17:00:00,9,Toscana,1628,246,1874,46026,47900,1586,2507,20319,1701,56467.0,13453.0,69920,1259794,822966.0
8,2020-11-11T17:00:00,10,Umbria,359,68,427,9907,10334,194,515,5701,218,4597.0,11656.0,16253,340853,194467.0
9,2020-11-11T17:00:00,11,Marche,536,74,610,10904,11514,625,701,7665,1077,20100.0,156.0,20256,353026,207157.0


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-10T17:00:00,1,Piemonte,4715,325,5040,53795,58835,2579,3659,40737,4742,44444.0,59870.0,104314,1178337,745706.0
1,2020-11-10T17:00:00,2,Valle d'Aosta,161,15,176,2123,2299,33,127,1946,218,3936.0,527.0,4463,46630,27907.0
2,2020-11-10T17:00:00,3,Lombardia,6682,708,7390,128115,135505,4046,10955,122410,18571,213059.0,63427.0,276486,3324961,2039672.0
3,2020-11-10T17:00:00,5,Veneto,1596,204,1800,48450,50250,-2912,2763,31362,2643,26311.0,57944.0,84255,2479307,962389.0
4,2020-11-10T17:00:00,6,Friuli Venezia Giulia,336,47,383,7458,7841,9,482,7213,476,13482.0,2048.0,15530,589577,245027.0
5,2020-11-10T17:00:00,7,Liguria,1315,92,1407,11929,13336,615,1172,22828,1976,26677.0,11463.0,38140,496323,253455.0
6,2020-11-10T17:00:00,8,Emilia-Romagna,2020,215,2235,39545,41780,2150,2430,29230,4845,51310.0,24545.0,75855,1763487,931259.0
7,2020-11-10T17:00:00,9,Toscana,1582,242,1824,44490,46314,1834,2223,19451,1648,54431.0,12982.0,67413,1241973,813816.0
8,2020-11-10T17:00:00,10,Umbria,363,66,429,9711,10140,144,485,5392,206,4484.0,11254.0,15738,336098,192401.0
9,2020-11-10T17:00:00,11,Marche,525,75,600,10289,10889,438,504,7599,1067,19405.0,150.0,19555,349370,205090.0


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-11T17:00:00,1,Piemonte,4790,340,5130,55224,60354,1519,2953,42087,4826,45698.0,61569.0,107267,1191915,755259.0
1,2020-11-11T17:00:00,2,Valle d'Aosta,160,17,177,2155,2332,33,125,2031,225,4052.0,536.0,4588,47373,28237.0
2,2020-11-11T17:00:00,3,Lombardia,6907,764,7671,134247,141918,6413,8180,124025,18723,221239.0,63427.0,284666,3377673,2066395.0
3,2020-11-11T17:00:00,4,Trentino-Alto Adige,734,68,802,10870,11672,200,639,13593,844,20825.0,5284.0,26109,579716,NaN
4,2020-11-11T17:00:00,5,Veneto,1669,208,1877,50358,52235,1985,3082,32413,2689,26526.0,60811.0,87337,2494150,965761.0
5,2020-11-11T17:00:00,6,Friuli Venezia Giulia,346,46,392,7773,8165,324,572,7448,489,13960.0,2142.0,16102,595748,246668.0
6,2020-11-11T17:00:00,7,Liguria,1315,98,1413,12542,13955,619,1102,23278,2009,27482.0,11760.0,39242,503245,256277.0
7,2020-11-11T17:00:00,8,Emilia-Romagna,2072,222,2294,41597,43891,2111,2428,29516,4876,52446.0,25837.0,78283,1784161,939211.0
8,2020-11-11T17:00:00,9,Toscana,1628,246,1874,46026,47900,1586,2507,20319,1701,56467.0,13453.0,69920,1259794,822966.0
9,2020-11-11T17:00:00,10,Umbria,359,68,427,9907,10334,194,515,5701,218,4597.0,11656.0,16253,340853,194467.0


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,0,2020-11-10T17:00:00,1,Piemonte,4715,325,5040,53795,58835,2579,3659,40737,4742,44444.0,59870.0,104314,1178337,745706.0
1,1,2020-11-10T17:00:00,2,Valle d'Aosta,161,15,176,2123,2299,33,127,1946,218,3936.0,527.0,4463,46630,27907.0
2,2,2020-11-10T17:00:00,3,Lombardia,6682,708,7390,128115,135505,4046,10955,122410,18571,213059.0,63427.0,276486,3324961,2039672.0
3,19,2020-11-10T17:00:00,4,Trentino-Alto Adige,717,65,782,10690,11472,84,618,13171,827,20337.0,5133.0,25470,572731,NaN
4,3,2020-11-10T17:00:00,5,Veneto,1596,204,1800,48450,50250,-2912,2763,31362,2643,26311.0,57944.0,84255,2479307,962389.0
5,4,2020-11-10T17:00:00,6,Friuli Venezia Giulia,336,47,383,7458,7841,9,482,7213,476,13482.0,2048.0,15530,589577,245027.0
6,5,2020-11-10T17:00:00,7,Liguria,1315,92,1407,11929,13336,615,1172,22828,1976,26677.0,11463.0,38140,496323,253455.0
7,6,2020-11-10T17:00:00,8,Emilia-Romagna,2020,215,2235,39545,41780,2150,2430,29230,4845,51310.0,24545.0,75855,1763487,931259.0
8,7,2020-11-10T17:00:00,9,Toscana,1582,242,1824,44490,46314,1834,2223,19451,1648,54431.0,12982.0,67413,1241973,813816.0
9,8,2020-11-10T17:00:00,10,Umbria,363,66,429,9711,10140,144,485,5392,206,4484.0,11254.0,15738,336098,192401.0


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione
0,2020-11-11T17:00:00,1,Piemonte,4790,340,5130,55224,60354,1519,2953,42087,4826,45698.0,61569.0,107267,1191915,755259.0,4341375
1,2020-11-11T17:00:00,2,Valle d'Aosta,160,17,177,2155,2332,33,125,2031,225,4052.0,536.0,4588,47373,28237.0,125501
2,2020-11-11T17:00:00,3,Lombardia,6907,764,7671,134247,141918,6413,8180,124025,18723,221239.0,63427.0,284666,3377673,2066395.0,10103969
3,2020-11-11T17:00:00,4,Trentino-Alto Adige,734,68,802,10870,11672,200,639,13593,844,20825.0,5284.0,26109,579716,NaN,1074819
4,2020-11-11T17:00:00,5,Veneto,1669,208,1877,50358,52235,1985,3082,32413,2689,26526.0,60811.0,87337,2494150,965761.0,4907704
5,2020-11-11T17:00:00,6,Friuli Venezia Giulia,346,46,392,7773,8165,324,572,7448,489,13960.0,2142.0,16102,595748,246668.0,1211357
6,2020-11-11T17:00:00,7,Liguria,1315,98,1413,12542,13955,619,1102,23278,2009,27482.0,11760.0,39242,503245,256277.0,1543127
7,2020-11-11T17:00:00,8,Emilia-Romagna,2072,222,2294,41597,43891,2111,2428,29516,4876,52446.0,25837.0,78283,1784161,939211.0,4467118
8,2020-11-11T17:00:00,9,Toscana,1628,246,1874,46026,47900,1586,2507,20319,1701,56467.0,13453.0,69920,1259794,822966.0,3722729
9,2020-11-11T17:00:00,10,Umbria,359,68,427,9907,10334,194,515,5701,218,4597.0,11656.0,16253,340853,194467.0,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione,totale_posti_terapia_intensiva
0,2020-11-11T17:00:00,1,Piemonte,4790,340,5130,55224,60354,1519,2953,42087,4826,45698.0,61569.0,107267,1191915,755259.0,4341375,367
1,2020-11-11T17:00:00,2,Valle d'Aosta,160,17,177,2155,2332,33,125,2031,225,4052.0,536.0,4588,47373,28237.0,125501,20
2,2020-11-11T17:00:00,3,Lombardia,6907,764,7671,134247,141918,6413,8180,124025,18723,221239.0,63427.0,284666,3377673,2066395.0,10103969,983
3,2020-11-11T17:00:00,4,Trentino-Alto Adige,734,68,802,10870,11672,200,639,13593,844,20825.0,5284.0,26109,579716,NaN,1074819,106
4,2020-11-11T17:00:00,5,Veneto,1669,208,1877,50358,52235,1985,3082,32413,2689,26526.0,60811.0,87337,2494150,965761.0,4907704,825
5,2020-11-11T17:00:00,6,Friuli Venezia Giulia,346,46,392,7773,8165,324,572,7448,489,13960.0,2142.0,16102,595748,246668.0,1211357,175
6,2020-11-11T17:00:00,7,Liguria,1315,98,1413,12542,13955,619,1102,23278,2009,27482.0,11760.0,39242,503245,256277.0,1543127,209
7,2020-11-11T17:00:00,8,Emilia-Romagna,2072,222,2294,41597,43891,2111,2428,29516,4876,52446.0,25837.0,78283,1784161,939211.0,4467118,516
8,2020-11-11T17:00:00,9,Toscana,1628,246,1874,46026,47900,1586,2507,20319,1701,56467.0,13453.0,69920,1259794,822966.0,3722729,415
9,2020-11-11T17:00:00,10,Umbria,359,68,427,9907,10334,194,515,5701,218,4597.0,11656.0,16253,340853,194467.0,880285,70


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-11-11T17:00:00,3,Lombardia,6907,764,7671,134247,141918,6413,8180,...,77.72,6132,-2775,52712,26723.0,152,1615,15.52,0.08096,2.82
1,2020-11-11T17:00:00,15,Campania,2077,181,2258,74412,76670,2442,3166,...,42.39,2438,450,18446,15137.0,34,690,17.16,0.05472,1.66
2,2020-11-11T17:00:00,5,Veneto,1669,208,1877,50358,52235,1985,3082,...,25.21,1908,319,14843,3372.0,46,1051,20.76,0.06280,1.78
3,2020-11-11T17:00:00,1,Piemonte,4790,340,5130,55224,60354,1519,2953,...,92.64,1429,-706,13578,9553.0,84,1350,21.75,0.06802,2.47
4,2020-11-11T17:00:00,9,Toscana,1628,246,1874,46026,47900,1586,2507,...,59.28,1536,284,17821,9150.0,53,868,14.07,0.06734,1.88
5,2020-11-11T17:00:00,12,Lazio,2913,255,3168,54415,57583,1994,2479,...,34.14,1868,-129,26554,22127.0,38,447,9.34,0.04226,1.25
6,2020-11-11T17:00:00,8,Emilia-Romagna,2072,222,2294,41597,43891,2111,2428,...,43.02,2052,-2,20674,7952.0,31,286,11.74,0.05435,1.75
7,2020-11-11T17:00:00,19,Sicilia,1376,202,1578,21986,23564,732,1487,...,37.55,697,286,9839,5956.0,27,728,15.11,0.02993,0.70
8,2020-11-11T17:00:00,16,Puglia,1131,142,1273,18826,20099,1213,1332,...,38.80,1100,87,7913,4836.0,27,92,16.83,0.03323,0.72
9,2020-11-11T17:00:00,7,Liguria,1315,98,1413,12542,13955,619,1102,...,46.89,613,-70,6922,2822.0,33,450,15.92,0.07141,2.54


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Umbria,68,2,70,97.14
1,Piemonte,340,27,367,92.64
2,Valle d'Aosta,17,3,20,85.00
3,Lombardia,764,219,983,77.72
4,Trentino-Alto Adige,68,38,106,64.15
5,Toscana,246,169,415,59.28
6,Marche,74,53,127,58.27
7,Liguria,98,111,209,46.89
8,Emilia-Romagna,222,294,516,43.02
9,Campania,181,246,427,42.39


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])) * 100).round(2)
    df['var_pos_perc'] = ((df['nuovi_positivi'] / (df['totale_positivi'] - df['nuovi_positivi'] + df['nuovi_dimessi_guariti'] + df['nuovi_deceduti'])) * 100).round(3)
    return df

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ./data/csv/regioni/covid19-piemonte.csv
Aggiornato ./data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ./data/csv/regioni/covid19-lombardia.csv
Aggiornato ./data/csv/regioni/covid19-veneto.csv
Aggiornato ./data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ./data/csv/regioni/covid19-liguria.csv
Aggiornato ./data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ./data/csv/regioni/covid19-toscana.csv
Aggiornato ./data/csv/regioni/covid19-umbria.csv
Aggiornato ./data/csv/regioni/covid19-marche.csv
Aggiornato ./data/csv/regioni/covid19-lazio.csv
Aggiornato ./data/csv/regioni/covid19-abruzzo.csv
Aggiornato ./data/csv/regioni/covid19-molise.csv
Aggiornato ./data/csv/regioni/covid19-campania.csv
Aggiornato ./data/csv/regioni/covid19-puglia.csv
Aggiornato ./data/csv/regioni/covid19-basilicata.csv
Aggiornato ./data/csv/regioni/covid19-calabria.csv
Aggiornato ./data/csv/regioni/covid19-sicilia.csv
Aggiornato ./data/csv/regioni/covid19-sardegna.csv
